In [240]:
import geopandas as gpd

df = gpd.read_file("../../data/vstupy_do_metra/vstupy_do_metra.geojson")

#prestupy = df[df["vst_linka"].str.contains(",", na=False)]
aggregated = df.groupby(["uzel_nazev"]).agg({
    "geometry": list,
    "vst_linka": lambda x: ",".join(sorted(set(",".join(x).replace(" ", "").split(","))))  # Get all unique lines
}).reset_index()


In [241]:
import plotly.graph_objects as go
import numpy as np
fig = go.Figure()

line_colors = {
    "A": "green",
    "B": "gold",
    "C": "red"
}

# For transfer station points (opaque)
color_to_rgba_points = {
    "green": "rgba(0, 128, 0, 0.9)",
    "gold": "rgba(255, 215, 0, 0.9)",
    "red": "rgba(255, 0, 0, 0.9)",
}
# For large circles (semi-transparent)
color_to_rgba_circles = {
    "green": "rgba(0, 128, 0, 0.2)",
    "gold": "rgba(255, 215, 0, 0.2)",
    "red": "rgba(255, 0, 0, 0.2)",
}

### Single Line Stations

In [242]:
for line, color in line_colors.items():
    df_line = df[df["vst_linka"] == line]
    if not df_line.empty:
        fig.add_trace(go.Scattermap(
            lon=df_line["geometry"].x,
            lat=df_line["geometry"].y,
            mode="markers",
            name=f"Line {line}",
            marker=dict(
                size=10,
                color=color,
                opacity=0.9
            ),
            text=df_line["vst_nazev"] + f" (Line {line})",
            hoverinfo="text"
        ))

### Transfer stations

In [243]:
def half_circle_scatter(_theta, _center_lon, _center_lat, _radius, _lat_scale, _circle_color, _fill_color, _name, _width=0):
    half_lons = _center_lon + (_radius / _lat_scale) * np.cos(_theta)
    half_lats = _center_lat + _radius * np.sin(_theta)

    # Close the half circle by adding center point
    half_lons = np.append(half_lons, [_center_lon])
    half_lats = np.append(half_lats, [_center_lat])


    return go.Scattermap(
        lon=half_lons.tolist(),
        lat=half_lats.tolist(),
        mode="lines",
        name=_name,
        showlegend=False,
        line=dict(
            width=_width,
            color=_circle_color,
        ),
        fill="toself",
        fillcolor=_fill_color,
        text=_name,
        hoverinfo="text"
    )

In [244]:
transfer_stations = df[df["vst_linka"].str.contains(",", na=False)]
point_radius = 0.0001

if not transfer_stations.empty:
    first_transfer_idx = transfer_stations.index[0]
    for idx, station in transfer_stations.iterrows():
        lines = station["vst_linka"].split(",")
        is_first = bool(idx == first_transfer_idx)
        center_lon_pt = station["geometry"].x
        center_lat_pt = station["geometry"].y
        lat_scale_pt = np.cos(np.radians(center_lat_pt))

        line1 = lines[0].strip()
        color1 = line_colors.get(line1)
        fill_color1 = color_to_rgba_points.get(color1)
        theta_half1 = np.linspace(0, np.pi, 25)

        fig.add_trace(half_circle_scatter(theta_half1, center_lon_pt, center_lat_pt,point_radius, lat_scale_pt, color1, fill_color1, station['uzel_nazev'], 1))

        line2 = lines[1].strip() if len(lines) > 1 else lines[0].strip()
        color2 = line_colors.get(line2)
        fill_color2 = color_to_rgba_points.get(color2)
        theta_half2 = np.linspace(np.pi, 2*np.pi, 25)

        fig.add_trace(half_circle_scatter(theta_half2, center_lon_pt, center_lat_pt,point_radius, lat_scale_pt, color2, fill_color2, station['uzel_nazev'], 1))

### Outer circles

In [245]:
for _, row in aggregated.iterrows():
    lons = [point.x for point in row["geometry"]]
    lats = [point.y for point in row["geometry"]]

    center_lat = sum(lats) / len(lats)
    center_lon = sum(lons) / len(lons)

    max_dist = 0
    lat_scale = np.cos(np.radians(center_lat))

    for lon, lat in zip(lons, lats):
        # Scale longitude difference to match latitude scale
        dist = np.sqrt(((lon - center_lon) * lat_scale)**2 + (lat - center_lat)**2)
        max_dist = max(max_dist, dist)

    radius = max_dist * 1.2

    # Determine color based on metro line(s)
    vst_linka_str = str(row['vst_linka']).strip()
    lines = [line.strip() for line in vst_linka_str.split(",")] if "," in vst_linka_str else [vst_linka_str]

    # For transfer stations with multiple lines, create half-and-half circles
    if len(lines) > 1:
        line1 = lines[0]
        circle_color1 = line_colors.get(line1)
        fill_color1 = color_to_rgba_circles.get(circle_color1)
        theta_half1 = np.linspace(0, np.pi, 50)

        fig.add_trace(half_circle_scatter(theta_half1, center_lon, center_lat, radius, lat_scale, circle_color1, fill_color1, row['uzel_nazev']))

        line2 = lines[1]
        circle_color2 = line_colors.get(line2)
        fill_color2 = color_to_rgba_circles.get(circle_color2)
        theta_half2 = np.linspace(np.pi, 2*np.pi, 50)

        fig.add_trace(half_circle_scatter(theta_half2, center_lon, center_lat, radius, lat_scale, circle_color2, fill_color2, row['uzel_nazev']))
    else:
        line = lines[0]
        circle_color = line_colors.get(line, "orange")
        fill_color = color_to_rgba_circles.get(circle_color, "rgba(255, 165, 0, 0.2)")
        theta = np.linspace(0, 2*np.pi, 100)

        fig.add_trace(half_circle_scatter(theta, center_lon, center_lat, radius, lat_scale, circle_color, fill_color, row['uzel_nazev']))


### Final layout

In [246]:


min_lat, max_lat = df["geometry"].y.min(),  df["geometry"].y.max()
min_lon, max_lon = df["geometry"].x.min(), df["geometry"].x.max()

center_lat = (min_lat + max_lat) / 2
center_lon = (min_lon + max_lon) / 2

fig.add_trace(go.Scattermap(
    lon=[],
    lat=[],
    mode="markers",
    showlegend=False
))

fig.update_layout(
    title="Vstupy do metra",
    map=dict(
        center=dict(lat=center_lat, lon=center_lon),
        zoom=10
    ),
    height=800,
)

fig.show()